# Experimente zur Erstellung eines Deep Learning Modells zur Klassifikation!

### Importieren der benötigten Bibliotheken

Hier werden alle notwendigen Bibliotheken und Module importiert, die für die Datenverarbeitung und das Modelltraining benötigt werden.

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf 
import tensorflow_hub as hub
import tensorflow_io as tfio
import sys
sys.path.append("../EchoWatch")

### Herunterladen des Datensatzes

Die Pumpendaten werden von einer externen Quelle heruntergeladen und lokal gespeichert. Das Dataset enthält Audiodateien von Maschinengeräuschen.

In [ ]:
_ = tf.keras.utils.get_file('pump.zip',
                        'https://zenodo.org/record/3678171/files/dev_data_pump.zip?download=1',
                        cache_dir='./',
                        cache_subdir='data',
                        extract=True)

### Hilfsfunktion zum Laden von WAV-Dateien

Die Funktion `load_wav_16k_mono` lädt eine WAV-Datei, konvertiert sie in einen Float-Tensor und resampled sie auf 16 kHz. Dies stellt sicher, dass alle Audiodateien im gleichen Format vorliegen.

In [ ]:
# Utility functions for loading audio files and making sure the sample rate is correct.

@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

### Definieren der Testdateipfade

Hier werden die Pfade für positive (normale) und negative (anomale) Testdateien definiert. Diese Dateien werden später verwendet, um das Audiosignal zu visualisieren und zu analysieren.

In [ ]:
neg_test_file = os.path.join('data', 'pump', 'test' , 'anomaly_id_00_00000000.wav')
pos_test_file = os.path.join('data', 'pump', 'test' , 'normal_id_00_00000000.wav')
pos_test_file

### Laden und Visualisieren der WAV-Dateien

Die Audiodateien werden geladen und ihre Wellenformen werden visualisiert. Dies gibt einen ersten Einblick in die Struktur der Audiosignale.

In [ ]:
wave = load_wav_16k_mono(pos_test_file)
nwave = load_wav_16k_mono(neg_test_file)

In [ ]:
plt.plot(nwave) # Blau
plt.plot(wave) # Orange
plt.show()

### Definieren des Datensatzpfads

Der Pfad zum Testdatensatz wird definiert, um den Zugriff auf die Audiodateien zu erleichtern.

In [ ]:
Datensatz = os.path.join('data', 'pump', 'test')
Datensatz

### Erstellen von Dateipfaden und zugehörigen Labels für den Datensatz

Für jede Audiodatei im Datensatz wird der Dateipfad und das zugehörige Label (normal oder anomal) erstellt. Dies erleichtert die weitere Verarbeitung und das Training des Modells.

In [ ]:
file_paths = [os.path.join(Datensatz, filename) for filename in os.listdir(Datensatz)]
labels = [1 if 'normal' in filename else 0 for filename in os.listdir(Datensatz)]

data = tf.data.Dataset.from_tensor_slices((file_paths, labels))

# Jetzt haben Sie das Dataset mit den Dateipfaden und den entsprechenden Labels

### Anzeigen eines zufälligen Eintrags aus dem Datensatz

Ein zufälliger Eintrag aus dem Datensatz wird angezeigt, um die Konsistenz und Struktur der Daten zu überprüfen.

In [ ]:
data.shuffle(10000).as_numpy_iterator().next()

### Anzeigen des gesamten Datensatzes

Der gesamte Datensatz wird angezeigt. Dies gibt einen Überblick über die Struktur des Datasets und die vorhandenen Daten.

In [ ]:
data

### Vorverarbeitung der Audiodaten
Die Funktion preprocess führt mehrere Schritte zur Vorverarbeitung von Audiodaten durch:

In [ ]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

### Anpassung und Auffüllen der Audiodaten
Das Audiosignal wird zuerst mit load_wav_16k_mono geladen und dann auf eine Länge von 160.000 Samples beschnitten. Falls das Signal kürzer ist, wird es mit Nullen aufgefüllt, um diese Länge zu erreichen.

In [ ]:
wav = load_wav_16k_mono(pos_test_file)
wav = wav[:160000]
zero_padding = tf.zeros([160000] - tf.shape(wav), dtype=tf.float32)
wav = tf.concat([zero_padding, wav],0)
wav

### Erstellung und Anpassung des Spektrogramms
Ein Spektrogramm des Audiosignals wird mit der Short-Time Fourier Transform (STFT) erstellt. Anschließend wird der Betrag des Spektrogramms genommen, um komplexe Werte in absolute Werte zu konvertieren. Zuletzt wird eine zusätzliche Dimension hinzugefügt, um das Spektrogramm für nachfolgende Verarbeitungsschritte vorzubereiten.

In [ ]:
spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
spectrogram = tf.abs(spectrogram)
spectrogram = tf.expand_dims(spectrogram, axis=2)

### Zufällige Auswahl von Daten
Ein zufälliges Datenpaar, bestehend aus Dateipfad und zugehörigem Label, wird aus dem Datensatz ausgewählt. Der Datensatz wird zuerst durchmischt (shuffle), wobei eine Puffergröße von 10.000 verwendet wird, um die Mischreihenfolge zu bestimmen. Anschließend wird das erste Element des durchmischten Datensatzes ausgewählt.

In [ ]:
filepath, label = data.shuffle(buffer_size=10000).as_numpy_iterator().next()

### Anwendung der Vorverarbeitung
Das zuvor ausgewählte Audiosignal (über den Dateipfad) und das zugehörige Label werden durch die preprocess Funktion verarbeitet. Das Ergebnis ist ein Spektrogramm des Audiosignals und das unveränderte Label.

In [ ]:
spectrogram, label = preprocess(filepath, label)

### Visualisierung des Spektrogramms
Das berechnete Spektrogramm wird visualisiert. Die Darstellung erfolgt in einem großformatigen Plot mit den Maßen 30x20. Das Spektrogramm wird transponiert, um es korrekt darzustellen, wobei nur der erste Kanal gezeigt wird.

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(tf.transpose(spectrogram)[0])
plt.show()

### Datenverarbeitung und Vorbereitung für das Training
Vorverarbeitung: Jedes Element im Datensatz wird mit der preprocess Funktion verarbeitet, um Spektrogramme zu erhalten.

Zwischenspeichern: Die Daten werden im Cache gespeichert, um das Laden während des Trainings zu beschleunigen.

Mischen: Der Datensatz wird durchmischt, wobei eine Puffergröße von 1.000 verwendet wird.

Batching: Die Daten werden in Batches von jeweils 16 Einträgen gruppiert, was für das Training von neuronalen Netzwerken üblich ist.

Vorabladen: Es werden 8 Batches im Voraus geladen, um die Datenbereitstellung während des Trainings zu optimieren.

In [ ]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(16)
data = data.prefetch(8)

### Aufteilen des Datensatzes in Trainings- und Testdaten
Der Datensatz wird in zwei Teile aufgeteilt:

Trainingsdaten: Die ersten 38 Batches des Datensatzes werden für das Training verwendet. (70%)

Testdaten: Die folgenden 16 Batches, die nach Überspringen der ersten 38 Batches kommen, werden als Testdatensatz verwendet. (30%)

In [ ]:
train = data.take(38)
test = data.skip(38).take(16)

### Laden des ersten Trainingsbatches
Das erste Batch von Trainingsdaten wird geladen. Es enthält sowohl die Spektrogramm-Samples als auch die zugehörigen Labels. Dies kann nützlich sein, um einen Überblick über die Daten zu bekommen oder sie weiter zu analysieren.

In [ ]:
samples, labels = train.as_numpy_iterator().next()

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
model = Sequential()
model.add(Conv2D(16, (9,9), activation='relu', input_shape=(1491, 257,1)))
model.add(Conv2D(16, (9,9), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (6,6), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2))) 
model.add(tf.keras.layers.Conv2D(16, (6,6), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))  # Zusätzliche Conv2D-Schicht
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.GlobalAveragePooling2D())  # Ersetzt Flatten
model.add(tf.keras.layers.Dropout(0.5))  # Hilft gegen Overfitting
model.add(tf.keras.layers.Dense(64, activation='relu'))  # Reduzierte Neuronenanzahl
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()
    
# Erste Conv2D Schicht
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))
    
# Zweite Conv2D Schicht
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))

# Dritte Conv2D Schicht
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))
    
# Dense Schichten
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()
    
# Erste Conv2D Schicht
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))
 
# Zweite Conv2D Schicht
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))

    # Dritte Conv2D Schicht
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))
    
    # Dense Schichten
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()
# Erste Conv2D Schicht
model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D(2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))
    
    # Zweite Conv2D Schicht
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))

    # Dritte Conv2D Schicht
model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2, strides=2))
model.add(tf.keras.layers.Dropout(0.3))

# Dense Schichten
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = Sequential([
    # Input Layer: The input shape is derived from your spectrogram shape
    # First Convolutional Layer
    Conv2D(16, (3, 3), activation='relu', input_shape=(spectrogram.shape[1], spectrogram.shape[2], 1)),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Second Convolutional Layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Third Convolutional Layer
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    BatchNormalization(),

    # Flatten Layer
    Flatten(),

    # Fully Connected Layer
    Dense(128, activation='relu'),
    Dropout(0.5),

    # Output Layer
    Dense(1, activation='sigmoid')
])

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
model = tf.keras.models.Sequential()

# Convolutional layer 1
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(1491, 257, 1)))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))

# Convolutional layer 2
model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))

# Convolutional layer 3
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.Dropout(0.3))

# Flatten and dense layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

### Erstellen des Modells

Ein neuronales Netzwerkmodell wird erstellt, um die Klassifikation durchzuführen.

In [ ]:
def create_more_compact_cnn_model(input_shape):
    model = tf.keras.models.Sequential()

    # Convolutional layer 1
    model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', strides=2, input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D(2))
    model.add(tf.keras.layers.Dropout(0.3))

    # Convolutional layer 2
    model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', strides=2))
    model.add(tf.keras.layers.MaxPooling2D(2))
    model.add(tf.keras.layers.Dropout(0.3))

    # Convolutional layer 3
    model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', strides=2))
    model.add(tf.keras.layers.MaxPooling2D(2))
    model.add(tf.keras.layers.Dropout(0.3))

    # Flatten and dense layers
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer='Adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    
    return model


### Kompilieren des Modells

Das Modell wird mit einem Optimierer, einem Verlust und Metriken kompiliert.

In [ ]:
model.compile('Adam', loss='BinaryCrossentropy', metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()]) #mit accuracy

### Anzeigen des Modells inklusive Parameter

In [ ]:
model.summary()

### Training des Modells

Das Modell wird mit den Trainingsdaten trainiert.

In [ ]:
hist = model.fit(train, epochs=20, validation_data=test) #Pro Epoche ca. 2min

### Visualisierung der Daten

Die Daten werden visualisiert, um Trends, Muster oder Anomalien zu erkennen.

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'b')
plt.show()

In [ ]:
plt.title('Precision')
plt.plot(hist.history['precision_14'], 'r')
plt.plot(hist.history['val_precision_14'], 'b')
plt.show()

In [ ]:
plt.title('Recall')
plt.plot(hist.history['recall_14'], 'r')
plt.plot(hist.history['val_recall_14'], 'b')
plt.show()

In [ ]:
plt.title('accuracy')
plt.plot(hist.history['accuracy'], 'r')
plt.plot(hist.history['val_accuracy'], 'b')
plt.show()

### Bewertung des Modells

Das trainierte Modell wird mit den Testdaten bewertet, um seine Leistung zu messen.

In [ ]:
test_loss, test_accuracy, test_recall, test_precision = model.evaluate(test)

### Anzeigen der Testergebnisse
Die Ergebnisse des Modelltests, einschließlich Verlust, Genauigkeit, Recall und Precision, werden ausgegeben.

In [ ]:
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Recall: {test_recall}")
print(f"Test Precision: {test_precision}")

### Evaluation mit den Trainingsdaten
Das Modell wird mit den Trainingsdaten bewertet, um den Verlust und die Genauigkeit zu ermitteln. Diese Metriken geben Aufschluss darüber, wie gut das Modell auf den Daten trainiert wurde, mit denen es auch trainiert wurde.

In [ ]:
loss, accuracy = model.evaluate(train)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

### Vorbereitung der Testdaten
Das erste Batch von Testdaten wird geladen, um sowohl die Eingabedaten (X_test) als auch die zugehörigen Labels (y_test) zu erhalten.

In [ ]:
X_test, y_test = test.as_numpy_iterator().next()

### Vorhersage mit dem Modell
Das trainierte Modell wird verwendet, um Vorhersagen für die Testdaten zu treffen. Die Vorhersagen sind kontinuierliche Werte, die anschließend in binäre Werte (1 oder 0) umgewandelt werden, basierend auf einem Schwellenwert von 0,5.

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat = [1 if prediction > 0.5 else 0 for prediction in yhat]

### Konvertierung der Testlabels
Die Testlabels (y_test) werden in den Integer-Datentyp konvertiert, um sie mit den binären Vorhersagen vergleichen zu können.

In [ ]:
y_test.astype(int)